In [1]:
#Import packages

#Rdkit
from rdkit import Chem

#Pandas
import pandas as pd
from pandas import DataFrame

#Others
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

import joblib
import random
import numpy as np
import os
import math
import warnings

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from plotnine import *

In [2]:
def print_RxC(dataframe):
    rows= dataframe.shape[0]
    columns= dataframe.shape[1]
    rows_columns=print(dataframe.shape[0],"Moleculars ×",dataframe.shape[1],"Descriptors")
    return rows_columns
def trimm_correlated(df_in, threshold):
    df_corr = df_in.corr(method='spearman')
    df_not_correlated = ~(df_corr.mask(np.tril(np.ones([len(df_corr)]*2, dtype=bool))).abs() > threshold).any()
    un_corr_idx = df_not_correlated.loc[df_not_correlated[df_not_correlated.index] == True].index
    df_out = df_in[un_corr_idx]
    return df_out

In [15]:
#Setting the Work Path
warnings.filterwarnings ('ignore')
#Work Path
os.chdir("C:\\Users\\A\\Desktop\\Bioactive\\VZV")

In [16]:
#AlvaDesc
from alvadesccliwrapper.alvadesc import AlvaDesc
#AlvaDesc Path(Linux default alvaDescCLI location)
aDesc = AlvaDesc('E:\\C++\\alvaDesc\\alvaDescCLI.exe') 

In [17]:
Pubchem_data=pd.read_csv("./Pubchem_data.csv",sep=',')
Iso_smiles=Pubchem_data['Iso_smiles']
print_RxC(Pubchem_data)
Pubchem_data.head()

11406 Moleculars × 3 Descriptors


,cid,Iso_smiles,Type
0,5273,C1CC(OC1)[N+]2=CNC3=C(N=CN=C32)N,Adenosine
1,13463,C1=NC(=C2C(=N1)N(C=N2)C3C(C(C(O3)CCl)O)O)N,Adenosine
2,39148,CN(C)C1[C@H](O[C@H](C1O)N2C=NC3=C(N=CN=C32)N)CO,Adenosine
3,56328,C1[C@H](O[C@H](C1F)N2C=NC3=C(N=CN=C32)N)CO,Adenosine
4,67358,C1=NC(=C2C(=N1)N(C=[N+]2[O-])[C@H]3[C@@H]([C@@...,Adenosine


In [18]:
Smis=[]
for i in Iso_smiles:
    Smis.append(i)
Mols = []
for Smi in Smis:
    Mol = Chem.MolFromSmiles(Smi)
    Mols.append(Mol)
Canonical_smis = []
for Mol in Mols:
    Canonical_smi = Chem.MolToSmiles(Mol)
    Canonical_smis.append(Canonical_smi)
Pubchem_data['Canonical_smiles']=Canonical_smis

In [19]:
#Load the data
Raw_data= pd.read_csv('./Original_data.csv',index_col=0)
X_scaled_data= pd.read_csv('./X_scaled_data.csv',index_col=0)
Transformer= joblib.load(filename='./Scaler_transformer.pkl')

In [20]:
Pubchem_data=Pubchem_data.drop_duplicates(subset="cid")
Pubchem_data=Pubchem_data.drop_duplicates(subset="Canonical_smiles")
Pubchem_data=Pubchem_data[~ Pubchem_data['Canonical_smiles'].isin(Raw_data['Canonical_smiles'])]
Pubchem_data.index=Pubchem_data['cid']
Pubchem_data.drop('cid',axis = 1,inplace = True)
print_RxC(Pubchem_data)
Pubchem_data.to_csv("./Pubchem_data_clearn.csv",sep=',',header=1,index=1)
Pubchem_data.head()

7236 Moleculars × 3 Descriptors


,Iso_smiles,Type,Canonical_smiles
cid,,,
5273,C1CC(OC1)[N+]2=CNC3=C(N=CN=C32)N,Adenosine,Nc1ncnc2c1[nH]c[n+]2C1CCCO1
13463,C1=NC(=C2C(=N1)N(C=N2)C3C(C(C(O3)CCl)O)O)N,Adenosine,Nc1ncnc2c1ncn2C1OC(CCl)C(O)C1O
39148,CN(C)C1[C@H](O[C@H](C1O)N2C=NC3=C(N=CN=C32)N)CO,Adenosine,CN(C)C1C(O)[C@H](n2cnc3c(N)ncnc32)O[C@@H]1CO
56328,C1[C@H](O[C@H](C1F)N2C=NC3=C(N=CN=C32)N)CO,Adenosine,Nc1ncnc2c1ncn2[C@@H]1O[C@H](CO)CC1F
67358,C1=NC(=C2C(=N1)N(C=[N+]2[O-])[C@H]3[C@@H]([C@@...,Adenosine,Nc1ncnc2c1[n+]([O-])cn2[C@@H]1O[C@H](CO)[C@@H]...


In [21]:
X_scaled_data

,s4_pathLength,maxDistfromCC,meanDistFromCC,s4_size,s4_numRotBonds,chiralMoment,s4_relPathLength,s4_relSize,s34_size,SHED_DL,...,piPC10,GATS8p,SM08_EA(bo),MATS7i,VE1sign_H2,CATS2D_04_AP,F05[N-O],NsssN,nCt,nRCOOR
cid,,,,,,,,,,,,,,,,,,,,,
3011383,0.187500,0.187500,0.230070,0.161290,0.047619,0.150272,0.473684,0.574163,0.194574,0.206340,...,0.521670,0.302063,0.572597,0.553284,0.738502,0.75,0.090909,0.000000,0.000000,0.0
53357718,0.250000,0.250000,0.268196,0.209677,0.071429,0.182216,0.461538,0.545455,0.243218,0.484777,...,0.708258,0.301601,0.798497,0.591377,0.166918,0.50,0.181818,0.000000,0.000000,0.0
135398513,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.543194,0.452865,0.617059,0.595804,0.270720,0.50,0.090909,0.000000,0.000000,0.0
446727,0.229167,0.229167,0.239808,0.198925,0.039683,0.184936,0.578947,0.708134,0.194574,0.462054,...,0.498819,0.388422,0.432780,0.609791,0.239609,0.00,0.090909,0.142857,0.000000,0.0
168280858,0.187500,0.187500,0.166683,0.193548,0.023810,0.169144,0.310345,0.451411,0.218896,0.156377,...,0.648886,0.278270,0.692947,0.398806,0.004063,0.50,0.181818,0.142857,0.142857,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60167465,0.322917,0.322917,0.310744,0.263441,0.015873,0.250500,0.673913,0.774704,0.243218,0.610049,...,0.719420,0.387609,0.523241,0.547944,0.257310,0.00,0.090909,0.142857,0.142857,0.0
60167322,0.385417,0.385417,0.382607,0.327957,0.015873,0.317016,0.685185,0.821549,0.291862,0.712393,...,0.831949,0.376193,0.618902,0.589204,0.269661,0.00,0.090909,0.142857,0.000000,0.0
70683866,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.706734,...,0.840226,0.340897,0.809895,0.581516,0.214488,0.50,0.181818,0.000000,0.000000,0.0


In [22]:
Canonical_smis=Pubchem_data['Canonical_smiles']

In [23]:
#Calculate the descriotors
aDesc.set_input_SMILES(Canonical_smis.tolist())
if not aDesc.calculate_descriptors('ALL'):
 print('Error: ' + aDesc.get_error())
else:
 Data_all=aDesc.get_output()

In [24]:
Data_all=pd.DataFrame(Data_all)
Desc_names_names=aDesc.get_output_descriptors()
Data_all.columns=Desc_names_names
Data_all.index=Pubchem_data.index

In [25]:
X_test=Data_all[X_scaled_data.columns]
X_test.head()

,s4_pathLength,maxDistfromCC,meanDistFromCC,s4_size,s4_numRotBonds,chiralMoment,s4_relPathLength,s4_relSize,s34_size,SHED_DL,...,piPC10,GATS8p,SM08_EA(bo),MATS7i,VE1sign_H2,CATS2D_04_AP,F05[N-O],NsssN,nCt,nRCOOR
cid,,,,,,,,,,,,,,,,,,,,,
5273,5.00,5.00,2.714286,10.000000,0.000000,15.362291,0.333333,0.666667,12.0,4.0,...,6.131456,0.651121,12.301158,-0.326495,0.067921,3.0,0.0,0.0,0.0,0.0
13463,6.25,6.25,3.333333,11.750000,0.750000,17.966397,0.328947,0.618421,16.0,3.0,...,6.661040,0.954296,12.339780,-0.113302,0.037320,2.0,2.0,0.0,0.0,0.0
39148,6.25,6.25,3.300000,11.750000,0.750000,17.883487,0.297619,0.559524,17.5,0.0,...,6.813020,0.906288,12.352307,-0.287220,0.023403,2.0,3.0,1.0,0.0,0.0
56328,6.00,6.00,3.274510,11.333333,0.666667,17.259745,0.333333,0.629630,15.0,2.0,...,6.540697,1.104736,12.322769,0.053539,0.021332,2.0,1.0,0.0,0.0,0.0
67358,6.25,6.25,3.434211,12.750000,0.750000,19.586399,0.312500,0.637500,17.0,0.0,...,6.712087,1.066973,12.512895,-0.109247,0.070736,4.0,3.0,0.0,0.0,0.0


In [26]:
X_test_scaled=Transformer.transform(X_test)
X_test_scaled =pd.DataFrame(X_test_scaled)
X_test_scaled.columns=X_test.columns
X_test_scaled.index=X_test.index
X_test_scaled.to_csv("./X_test_data.csv",sep=',',header=1,index=1)
print_RxC(X_test_scaled)
X_test_scaled.head()

7236 Moleculars × 1422 Descriptors


,s4_pathLength,maxDistfromCC,meanDistFromCC,s4_size,s4_numRotBonds,chiralMoment,s4_relPathLength,s4_relSize,s34_size,SHED_DL,...,piPC10,GATS8p,SM08_EA(bo),MATS7i,VE1sign_H2,CATS2D_04_AP,F05[N-O],NsssN,nCt,nRCOOR
cid,,,,,,,,,,,,,,,,,,,,,
5273,0.156250,0.156250,0.178422,0.161290,0.000000,0.150272,0.500000,0.727273,0.145931,0.217814,...,0.684852,0.202068,0.750114,0.193889,0.177832,0.75,0.000000,0.000000,0.0,0.0
13463,0.195312,0.195312,0.219114,0.189516,0.017857,0.175745,0.493421,0.674641,0.194574,0.163361,...,0.744004,0.296156,0.758093,0.453199,0.097712,0.50,0.181818,0.000000,0.0,0.0
39148,0.195312,0.195312,0.216923,0.189516,0.017857,0.174934,0.446429,0.610390,0.212816,0.000000,...,0.760979,0.281257,0.760682,0.241660,0.061275,0.50,0.272727,0.142857,0.0,0.0
56328,0.187500,0.187500,0.215247,0.182796,0.015873,0.168832,0.500000,0.686869,0.182413,0.108907,...,0.730562,0.342843,0.754579,0.656130,0.055852,0.50,0.090909,0.000000,0.0,0.0
67358,0.195312,0.195312,0.225745,0.205645,0.017857,0.191591,0.468750,0.695455,0.206735,0.000000,...,0.749705,0.331123,0.793861,0.458131,0.185202,1.00,0.272727,0.000000,0.0,0.0
